Step 1: build a surface water network. You can "pickle" this, so it doesn't need to be repeated.

n = swn.SurfaceWaterNetwork.from_lines(gdf.geometry)

n.to_pickle("surface-water-network.pkl")

# then in a later session, skip the above and just do:

n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

Step 2: load a MF6 model, then find the intersections:

sim = flopy.mf6.MFSimulation.load(...)

m = sim.get_model(...)

nm = swn.SwnMf6.from_swn_flopy(n, m)

Most of the results will be in the nm.reaches property, but other reach datasets will need to be specified, including "man", "rbth", "rgrd", "rhk", "rtp", and "rwid". There are two methods to format the PACKAGEDATA:

nm.flopy_packagedata

nm.write_packagedata("packagedata.dat")

Similar with CONNECTIONDATA:

nm.flopy_connectiondata

nm.write_connectiondata("connectiondata.dat")

There are a few "helper" methods to sort out things like "set_reach_slope" based on a few methods. One missing one is "set_reach_elevation" or whatever to make the reaches fit in the layer and/or move the layer elevations to fit the stream.

And lastly, there is no PERIOD data yet. I'm working on this, which is holding up the merge.


In [3]:
import geopandas
import os
import swn
import flopy
import numpy as np
import time

# SW network pickle

#only do this once... takes forever... then load pickle
gdb_dir = 'D:\modelling\data'
gdb_fname = 'nzRec2_v5.gdb'
gdb_path = os.path.join(gdb_dir, gdb_fname)
# Read national data of streams
gdf_lines = geopandas.read_file(gdb_path, layer='riverlines')
gdf_lines.set_index('nzsegment', inplace=True, verify_integrity=True)
gdf_ws = geopandas.read_file(gdb_path, layer='rec2ws')
gdf_ws.set_index('nzsegment', inplace=True, verify_integrity=True)
# Convert MultiLineString -> LineString
lines = gdf_lines.geometry.apply(lambda x: x.geoms[0]) #what is geoms[0]
polygons = gdf_ws.geometry.apply(lambda x: x.geoms[0])
#ni_lines = gdf_lines.loc[gdf_lines.index < 10000000, "geometry"]
# requires reindex otherwise failure in core.from_lines
t0=time.time()
n = swn.SurfaceWaterNetwork.from_lines(lines,polygons.reindex(index=lines.index))
print(time.time()-t0)
n.to_pickle("surface-water-network.pkl")

# get the pickle

In [5]:
n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

# Load MF6 model

In [6]:
os.getcwd()

'D:\\modelling\\surface-water-network'

In [7]:
sim_ws=os.path.join('..','zmodels','20210614_simulation','si_5lay')
model_name='si_5lay'
sim=flopy.mf6.MFSimulation.load(sim_ws=sim_ws)
gwf=sim.get_model(model_name)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package sto...
    loading package rch...
    loading package ic...
    loading package ghb...
    loading package drn...
    loading package npf...
    loading package oc...
  loading ims package si_5lay...


## spatial reference for model

In [8]:
#sr=flopy.utils.reference.SpatialReference.from_gridspec(os.path.join(sim_ws,model_name+'.grid.spc'))
#gwf.dis.xorigin=sr.xul
#gwf.dis.yorigin=sr.yul-np.sum(gwf.dis.delr.data)
#gwf.dis.write()

In [9]:
# this also takes forever
t0=time.time()
ngwf = swn.SwnMf6.from_swn_flopy(n, gwf)
print(time.time()-t0)

12:12:49.714:WARNING:SwnMf6:failed to merge segnum 10000453 at (182, 1153): MULTILINESTRING ((1565024.201245195 5498640, 1564996.6582 5498667.5413, 1564960 5498667.540046895), (1565491.4141 5498697.5434, 1565440 5498697.541636366, 1565386.4656 5498697.5398, 1565356.4813 5498667.5535, 1565326.4961 5498667.5525, 1565298.945437673 5498640))
12:12:50.189:WARNING:SwnMf6:failed to merge segnum 10000432 at (183, 1150): MULTILINESTRING ((1563974.683888874 5498640, 1564000 5498614.68553518), (1564000 5498360.500746587, 1563917.2027 5498277.6982, 1563917.2036 5498247.713, 1563977.1759 5498187.7445, 1563977.176732745 5498160, 1563977.1768 5498157.7593))
12:12:53.021:WARNING:SwnMf6:failed to merge segnum 10000603 at (188, 1126): MULTILINESTRING ((1552191.897978832 5495760, 1552163.1826 5495788.7149, 1552103.2135 5495788.7144, 1552074.499578816 5495760), (1552522.9941 5496118.5485, 1552480 5496075.553539666, 1552463.0256 5496058.5788, 1552463.0259 5496028.5942, 1552433.0416 5495998.6094, 1552433.04

12:13:25.836:WARNING:SwnMf6:failed to merge segnum 10001993 at (220, 1214): MULTILINESTRING ((1594204.0984 5480826.5006, 1594240 5480826.501617629, 1594264.0738 5480826.5023, 1594294.0607 5480856.4908, 1594384.0239 5480856.4934, 1594407.529089068 5480880), (1594660.401975079 5480880, 1594720 5480820.405419829))
12:13:34.570:WARNING:SwnMf6:failed to merge segnum 12002266 at (223, 1090): MULTILINESTRING ((1535200 5479409.214564622, 1535169.217644619 5479440, 1535131.6853 5479477.5361, 1535131.6868 5479507.5205), (1535169.44349358 5478960, 1535161.6439 5478967.8004, 1535161.6454 5478997.7847, 1535191.6313 5479027.7676, 1535191.6359 5479117.7207, 1535200 5479126.083963196))
12:13:46.002:WARNING:SwnMf6:failed to merge segnum 10002779 at (229, 1173): MULTILINESTRING ((1574517.5738 5476478.0722, 1574560 5476478.0722, 1574592.5405 5476478.0722, 1574622.5271 5476508.0589, 1574682.5005 5476508.0589, 1574712.4872 5476538.0456, 1574742.4739 5476538.0456, 1574764.428226786 5476560), (1574930.426363

12:14:12.786:WARNING:SwnMf6:failed to merge segnum 10003876 at (240, 1129): MULTILINESTRING ((1553572.1169 5470766.0832, 1553572.118181911 5470800, 1553572.1186 5470811.0618, 1553542.134 5470841.0487, 1553542.1351 5470871.0344, 1553512.1505 5470901.0212, 1553512.1516 5470931.007, 1553440 5471003.16381362), (1553440 5471158.706894246, 1553512.1625 5471230.8641, 1553512.164302506 5471280, 1553512.1647 5471290.8356))
12:14:13.341:WARNING:SwnMf6:failed to merge segnum 10003958 at (240, 1219): MULTILINESTRING ((1596640 5470833.882033786, 1596663.3017 5470810.5808, 1596693.2899 5470810.5811, 1596703.871211708 5470800), (1596892.623747038 5470800, 1596903.2068 5470810.5833, 1597113.1238 5470810.5855, 1597120 5470803.709449042, 1597121.854764723 5470801.854724521))
12:14:26.166:WARNING:SwnMf6:failed to merge segnum 10004267 at (245, 1171): MULTILINESTRING ((1573681.236912014 5468400, 1573692.8716 5468411.6343, 1573692.8725 5468471.6081, 1573662.8861 5468501.5954, 1573662.8871 5468561.5692, 157

12:16:01.776:WARNING:SwnMf6:failed to merge segnum 12008089 at (282, 1088): MULTILINESTRING ((1533760 5450802.17260972, 1533780.2111 5450781.9576, 1533780.19721309 5450640, 1533780.1935 5450602.0433), (1533760 5450952.115712545, 1533780.2257 5450931.8861, 1533780.2198 5450871.9147, 1533760 5450851.698810647), (1533818.433032226 5451120, 1533760 5451061.578308091))
12:16:11.411:WARNING:SwnMf6:failed to merge segnum 10008457 at (285, 1231): MULTILINESTRING ((1602400 5449278.775688605, 1602420.9325 5449278.7752, 1602435.9266 5449263.7804), (1602420.9429 5449698.6197, 1602420.942403291 5449680, 1602420.9421 5449668.6308, 1602450.9303 5449638.6412, 1602450.9281 5449548.6745, 1602420.9385 5449518.6864, 1602420.9377 5449488.6975, 1602400 5449467.760777432))
12:16:16.204:WARNING:SwnMf6:failed to merge segnum 12008825 at (286, 1069): MULTILINESTRING ((1524928.137965626 5448720, 1524874.2822 5448773.8681, 1524844.2968 5448773.8715, 1524814.3148 5448803.8604, 1524784.3293 5448803.8639, 1524754.34

12:17:57.958:WARNING:SwnMf6:failed to merge segnum 10012454 at (310, 1225): MULTILINESTRING ((1599961.406 5437163.3286, 1599961.407711962 5437200, 1599961.4102 5437253.2955, 1599991.4005 5437283.2832, 1599991.4019 5437313.2721, 1600000 5437321.869425923), (1600000 5437454.625126473, 1599991.4088 5437463.2171, 1599991.4129 5437553.1841))
12:18:10.697:WARNING:SwnMf6:failed to merge segnum 12013115 at (315, 1142): MULTILINESTRING ((1559750.589712356 5435280, 1559717.1928 5435246.6086, 1559717.1903 5435216.6213, 1559687.2005 5435186.6364, 1559687.193 5435096.6743, 1559680 5435089.482499241, 1559657.2032 5435066.6895, 1559657.2007 5435036.7021), (1560047.0488 5435186.6067, 1560017.0639 5435216.5965, 1559987.0766 5435216.599, 1559923.686029477 5435280))
12:18:14.134:WARNING:SwnMf6:failed to merge segnum 11012891 at (315, 1361): MULTILINESTRING ((1664800 5434972.52498044, 1664801.0052 5434971.5197, 1664830.9975 5434971.5184, 1664860.9886 5434941.5249, 1665010.9502 5434941.5189, 1665040.9413 5

...

22:18:30.246:WARNING:SwnMf6:failed to merge segnum 14312177 at (1507, 713): MULTILINESTRING ((1354240 4862827.144117522, 1354215.4129 4862802.4585, 1354215.713 4862652.4161, 1354203.346451258 4862640, 1354185.7645 4862622.3476, 1354185.8246 4862592.3392), (1354197.286883895 4863120, 1354240 4863077.457110947))
22:18:44.750:WARNING:SwnMf6:failed to merge segnum 15312403 at (1508, 552): MULTILINESTRING ((1276453.3504 4862387.2972, 1276480 4862414.059196966, 1276704.991880551 4862640), (1276951.878395204 4862640, 1276960 4862631.912483192))
22:19:28.808:WARNING:SwnMf6:failed to merge segnum 15312728 at (1509, 616): MULTILINESTRING ((1307334.9194 4861746.3411, 1307304.9615 4861716.2583, 1307274.9411 4861716.196, 1307214.7756 4861776.112, 1307214.7132 4861806.1324, 1307200 4861820.784585907), (1307200 4861971.855861227, 1307214.3392 4861986.2547, 1307214.2769 4862016.2751, 1307244.235 4862046.3578, 1307244.1727 4862076.3782, 1307274.1308 4862106.4609, 1307274.0685 4862136.4812, 1307250.4519

22:26:18.284:WARNING:SwnMf6:failed to merge segnum 15314618 at (1522, 461): MULTILINESTRING ((1232800 4855683.507938941, 1232812.8841 4855670.6805, 1232812.9502 4855640.6405, 1232843.0564 4855610.6666, 1232843.1226 4855580.6266, 1232984.370531288 4855440), (1233182.699671947 4855440, 1233203.8672 4855461.2611, 1233280 4855461.428927257, 1233354.0667 4855461.5922))
22:26:23.790:WARNING:SwnMf6:failed to merge segnum 15314075 at (1522, 534): MULTILINESTRING ((1267840 4855681.443718798, 1267864.2568 4855657.2921, 1267864.3221 4855627.2615, 1267894.4179 4855597.2962, 1267894.4832 4855567.2657, 1267924.5791 4855537.3005, 1267924.790756302 4855440, 1267924.8404 4855417.1783), (1267886.100944212 4855920, 1267863.7348 4855897.5366, 1267863.8001 4855867.506, 1267840 4855843.602329448))
22:26:33.379:WARNING:SwnMf6:failed to merge segnum 15313936 at (1520, 459): MULTILINESTRING ((1232320 4856711.053324891, 1232180.0597 4856570.4976, 1232180.1916 4856510.4171, 1232210.2978 4856480.4429, 1232210.363

22:38:20.199:WARNING:SwnMf6:failed to merge segnum 14318230 at (1551, 691): MULTILINESTRING ((1343200 4841579.146854191, 1343246.5829 4841532.7684), (1343245.4952 4842027.9358, 1343245.556513754 4842000, 1343245.594 4841982.9205, 1343215.6497 4841952.8445, 1343215.7156 4841922.8343, 1343200 4841907.049580184))
22:38:27.814:WARNING:SwnMf6:failed to merge segnum 15318316 at (1551, 589): MULTILINESTRING ((1294252.560940746 4841520, 1294240 4841532.504003424, 1294229.2776 4841543.1778), (1294493.293024694 4841520, 1294469.4617 4841543.7233, 1294349.3697 4841543.4505, 1294326.02549846 4841520))
22:38:44.753:WARNING:SwnMf6:failed to merge segnum 14318156 at (1552, 643): MULTILINESTRING ((1320640 4841489.353225, 1320616.6536 4841512.5954, 1320616.637047234 4841520, 1320616.5194 4841572.6275), (1320640 4841100.014001936, 1320617.5267 4841122.3869, 1320617.4595 4841152.403, 1320587.3762 4841182.3518, 1320587.3091 4841212.3678, 1320557.2258 4841242.3167, 1320557.0915 4841302.3488, 1320587.0404 4

22:45:27.937:WARNING:SwnMf6:failed to merge segnum 15320562 at (1583, 618): MULTILINESTRING ((1308154.419034634 4826634.445594497, 1308160 4826628.891188995, 1308163.7636 4826625.1455, 1308283.8384 4826625.432, 1308298.337022347 4826640), (1308509.925478495 4826640, 1308523.9875 4826626.0049, 1308640 4826626.28167617, 1308704.099 4826626.4346))
22:46:12.666:WARNING:SwnMf6:failed to merge segnum 15000071 at (1610, 391): MULTILINESTRING ((1199200 4813492.258213768, 1199224.8191 4813467.5687, 1199200 4813442.619292209), (1199269.2234 4813723.0976, 1199226.350584563 4813680, 1199224.2687 4813677.9072, 1199224.4259 4813617.8104, 1199200 4813593.256378882))
22:46:14.036:WARNING:SwnMf6:failed to merge segnum 15000086 at (1611, 405): MULTILINESTRING ((1206181.218717545 4813200, 1206106.6139 4813125.004, 1206046.5211 4813124.8469, 1206016.3961 4813154.8147, 1205956.3032 4813154.6576, 1205926.1781 4813184.6255, 1205920 4813184.609338429, 1205896.1316 4813184.5469, 1205880.597490586 4813200), (12

22:54:33.433:WARNING:SwnMf6:failed to merge segnum 15002856 at (1695, 389): MULTILINESTRING ((1198240 4772407.302103483, 1198247.346067684 4772400, 1198288.6232 4772358.9699), (1198257.584 4772689.4038, 1198257.8543 4772599.2608, 1198240 4772581.299073227))
22:55:30.747:WARNING:SwnMf6:failed to merge segnum 15003083 at (1701, 382): MULTILINESTRING ((1194925.25369144 4770000, 1194900.3439 4769974.939), (1195367.312579346 4769992.643043555, 1195360 4769985.286087111, 1195351.0709 4769976.3028, 1195110.6834 4769975.5755, 1195086.110637031 4770000))
22:55:31.567:WARNING:SwnMf6:failed to merge segnum 15003019 at (1700, 397): MULTILINESTRING ((1202289.2963542 4770000, 1202261.8899 4770027.2414, 1202231.8434 4770027.1506, 1202201.7062 4770057.1064, 1202171.6597 4770057.0156, 1202081.248 4770146.883, 1202081.1573 4770176.9295, 1202080 4770178.07983405), (1202080 4770447.004834373, 1202080.3413 4770447.3482, 1202080.242817085 4770480, 1202080.121408543 4770520.252742618))
22:55:33.058:WARNING:S

In [10]:
# https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-sfr.html?highlight=ustrf#block-packagedata
# started from sagehen example, tweaked
# can do ngwf.default_packagedata() now?
d={'rwid':10.0,'rbth':0.5,'man':0.04,'ustrf':1.0,'ndv': 0}
for k in ["man", "rbth", "rwid"]:
    ngwf.reaches[k]=d[k]

# was taking much time to fail with no zcoord, now takes 1.2 sec
t0=time.time()
#zcoord_ab or grid_top
try:
    ngwf.set_reach_slope(method='zcoord_ab')
except:
    print(time.time()-t0)


In [22]:
# check for nans, should have been fixed, taken care of
ngwf.set_reach_slope(method='grid_top')

14:10:08.728:WARNING:SwnMf6:enforcing min_slope for 4730 reaches (0.65%)


In [11]:
ngwf.set_reach_data_from_array('rtp',gwf.dis.top.array)

In [12]:
ngwf.set_reach_data_from_array('rhk',gwf.npf.k.array[0])

In [13]:
ngwf.reaches

,geometry,segnum,segndist,k,i,j,iseg,ireach,rlen,to_rno,from_rnos,to_div,ustrf,man,rbth,rwid,rtp,rhk
rno,,,,,,,,,,,,,,,,,,
1,"LINESTRING (1252079.983 5069265.748, 1252000.0...",14184033,0.297387,0,1077,500,1,1,584.888888,2,{},0,1.0,0.04,0.5,10.0,2045.372691,0.000124
2,"LINESTRING (1251520.000 5069205.447, 1251163.9...",14184033,0.797387,0,1077,499,1,2,398.489930,373647,{1},0,1.0,0.04,0.5,10.0,1999.955290,0.000099
3,"LINESTRING (1251268.821 5069881.099, 1251040.0...",14183716,0.183727,0,1076,499,2,1,323.736143,4,{},0,1.0,0.04,0.5,10.0,2060.988046,0.000091
4,"LINESTRING (1251040.000 5069652.088, 1251013.6...",14183716,0.524528,0,1076,498,2,2,276.772240,5,{3},0,1.0,0.04,0.5,10.0,2028.840800,0.000069
5,"LINESTRING (1250817.957 5069520.000, 1250743.4...",14183716,0.840801,0,1077,498,2,3,280.516351,373650,{4},0,1.0,0.04,0.5,10.0,1959.784528,0.000076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731612,"LINESTRING (1538051.063 5139612.919, 1538080.0...",13300013,0.619345,0,930,1097,323858,2,565.460921,731614,{731611},0,1.0,0.04,0.5,10.0,1.709218,41.473612
731613,"LINESTRING (1687548.326 5404902.159, 1687840.0...",11100009,0.256128,0,378,1408,323859,1,310.992841,0,{731597},0,1.0,0.04,0.5,10.0,0.813448,20.443768
731614,"LINESTRING (1538565.311 5139848.050, 1538933.0...",13300015,0.306655,0,930,1098,323860,1,434.767368,731615,"{731612, 551015}",0,1.0,0.04,0.5,10.0,1.204094,41.782822


In [14]:
n.segments

,geometry,to_segnum,from_segnums,cat_group,num_to_outlet,dist_to_outlet,sequence,stream_order,upstream_length,upstream_area,width
nzsegment,,,,,,,,,,,
1000005,"LINESTRING (1601528.857 6192580.400, 1601573.9...",1000007,{},1000015,3,2014.828119,288849,1,2.135679e+02,2.176544e+05,1.872521
1000003,"LINESTRING (1601782.856 6192881.076, 1601783.2...",1000007,{},1000015,3,2383.050130,288397,1,5.817899e+02,4.559958e+05,2.084752
1000008,"LINESTRING (1599355.244 6192363.831, 1599355.3...",1000010,{},1000010,2,1030.124265,293851,1,6.944680e+02,4.613857e+05,2.088826
1000010,"LINESTRING (1599237.075 6191778.666, 1599207.0...",0,"{1000008, 1000011}",1000010,1,335.656220,442459,2,3.340765e+03,2.330307e+06,2.972583
1000006,"LINESTRING (1602683.557 6192553.928, 1602684.0...",0,{},1000006,1,1205.990464,297106,1,1.205990e+03,8.634214e+05,2.346480
...,...,...,...,...,...,...,...,...,...,...,...
4170001,"LINESTRING (1975828.498 5785372.292, 1975437.9...",0,"{4080400, 4080785, 4080786}",4170001,1,542.278140,589186,6,1.761362e+06,1.101287e+09,39.595802
1040001,"LINESTRING (1731848.082 6017486.385, 1731928.7...",0,{1025403},1040001,1,1675.015974,592689,5,3.363715e+05,1.973785e+08,17.035500
7260002,"LINESTRING (1789499.342 5528919.920, 1789362.8...",0,"{7260001, 7238650}",7260002,1,156.868127,579943,5,8.847122e+04,5.386487e+07,9.368385


In [26]:
n.segments.to_csv(os.path.join(sim_ws,'znseg_mf6rch.csv'))

In [16]:
mask=[s for s in n.segments.index if (len(n.segments.loc[s,'from_segnums'])==0) & (n.segments.loc[s,'to_segnum']==0)]

In [17]:
len(mask)

6983

In [18]:
n.segments.drop(mask,inplace=True)

In [19]:
len(n.segments)

586534

In [20]:
help(n.remove)

Help on method remove in module swn.core:

remove(condition=False, segnums=[]) method of swn.core.SurfaceWaterNetwork instance
    Remove segments (and catchments).
    
    Parameters
    ----------
    condition : bool or pandas.Series
        Series of bool for each segment index, where True is to remove.
        Combined with 'segnums'. Defaut False (keep all).
    segnums : list
        List of segnums to remove. Combined with 'condition'. Default [].
    
    Returns
    -------
    None



In [23]:


# There are two methods to format the PACKAGEDATA:
ngwf.flopy_packagedata
ngwf.write_packagedata(os.path.join(sim_ws,model_name+'.sfr.reach.dat'))

#Similar with CONNECTIONDATA:
ngwf.flopy_connectiondata
ngwf.write_connectiondata(os.path.join(sim_ws,model_name+'.sfr.connection.dat'))


In [24]:
sfr=flopy.mf6.ModflowGwfsfr(gwf,packagedata={'filename':model_name+'.sfr.reach.dat'},
                            connectiondata={'filename':model_name+'.sfr.connection.dat'},
                           nreaches=len(ngwf.reaches))
#gwf.register_package(sfr)
sfr.write()

In [ ]:
gwf.write()

In [ ]:
help(ngwf)

# write shapefile, but not sfr info from grid intersection

In [ ]:
swn.file.gdf_to_shapefile(n.segments, 'segments.shp')